In [ ]:
#matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from dataset.dataloader import *
import operators as ops
import os

CKPT_DIR = "./logs/checkpoints"

In [ ]:
class VGG8:
    def __init__(self):
        self.criterion = ops.SoftmaxWithLoss()
        self.conv_layers = [
            # Layer 1 (B, 1, 28, 28) -> (B, 32, 28, 28)
            ops.Conv2D(in_channels=1, out_channels=32, kernel_size=3, padding=1, stride=1, name="L1_C"),
            ops.ReLU(name="L1_R"),

            # Layer2 (B, 32, 28, 28) -> (B, 64, 14, 14)
            ops.Conv2D(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=1, name="L2_C"),
            ops.ReLU(name="L2_R"),
            ops.MaxPooling(kernel_size=2, stride=2, name="L2_M"),
            
            # Layer 3 (B, 64, 14, 14) -> (B, 64, 14, 14)
            ops.Conv2D(in_channels=64, out_channels=64, kernel_size=3, padding=1, stride=1, name="L3_C"),
            ops.ReLU(name="L3_R"),

            # Layer 4 (B, 64, 14, 14) -> (B, 128, 7, 7)
            ops.Conv2D(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=1, name="L4_C"),
            ops.ReLU(name="L4_R"),
            ops.MaxPooling(kernel_size=2, stride=2, name="L4_M"),

            # Layer 5 (B, 128, 7, 7) -> (B, 256, 7, 7)
            ops.Conv2D(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=1, name="L5_C"),
            ops.ReLU(name="L5_R"),

            # Layer 6 (B, 256, 7, 7) -> (B, 256, 7, 7)
            ops.Conv2D(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1, name="L6_C"),
            ops.ReLU(name="L6_R")
        ]

        # Layer 7 (B, 256*7*7) -> (B, 256)
        self.fc_layers = [
            ops.FullyConnected(in_feature=256*7*7, out_feature=256, name="L7_FC"),
            ops.ReLU(name="L7_R"),

        # Layer 8 (B, 256) -> (B, 10)
            ops.FullyConnected(in_feature=256, out_feature=10, name="L8_FC")
        ]

    def backprop(self, pred: np.ndarray, label: np.ndarray, lr) -> None:
        # Backward
        #dout = self.criterion.backward(pred, label)
        dout = self.criterion.backward(1)
        for i in range(len(self.fc_layers)-1, -1, -1):
            dout = self.fc_layers[i].backward(dout)
        dout = dout.reshape(dout.shape[0], 256, 7, 7)
        for i in range(len(self.conv_layers)-1, -1, -1):
            dout = self.conv_layers[i].backward(dout)
        # Update
        for layer in self.conv_layers:
            layer.update(lr)
        for layer in self.fc_layers:
            layer.update(lr)
        return dout

    def forward(self, x: np.ndarray):
        i = 0
        for layer in self.conv_layers:
            i+=1
            x = layer.forward(x) 
            #print(x[0])
            if ((x[0] == x[1]).all() and (x[0] == x[-1]).all()):
                print(f"[{layer.name}] {x[0]}")
                break
        x = x.reshape(x.shape[0],-1)
        for layer in self.fc_layers:
            x = layer.forward(x)
            if ((x[0] == x[1]).all()):
                print(f"[{layer.name}]")
                break
        return x

    def save(self, fileName: str):
        with open(fileName, "wb") as f:
            pickle.dump(self, f)

    def load(self, fileName: str):
        with open(fileName, "rb") as f:
            self = pickle.load(f)

In [ ]:
model = VGG8()

In [ ]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

batch_size = 2
learning_rate = 0.01
loss_print_time = batch_size * 10
x_log = []
loss_log = []

print(f"Train:{x_train.shape}, Test:{x_test.shape}")
for i in x_train[0]:
    print(i)

In [ ]:

for epoch in range(10):
    accuracy = 0.0
    train_size = x_train.shape[0]
    running_loss = 0.0
    for i in tqdm(range(0, train_size, batch_size)):
        x = x_train[i:i+batch_size].reshape(batch_size, 1, 28, 28)   # B, C, H, W
        labels = t_train[i:i+batch_size]
        #optimizer.zero_grad()

        outputs = ops.Softmax(model.forward(x))
        #print(outputs)
        labels = labels
        loss = model.criterion.forward(outputs, labels)
        running_loss += loss
        model.backprop(outputs, labels, learning_rate)

        if (i % loss_print_time == 0):
            #running_loss /= loss_print_time
            print(f'[epoch:{epoch}, data num:{i}] loss: {running_loss}')
            x_log.append(i)
            loss_log.append(running_loss)
            running_loss = 0.0
            #model.save(os.path.join(CKPT_DIR, "epoch_%03d_%06d.pkl" % (epoch, i) ))




In [ ]:
plt.plot(x_log, loss_log)
plt.show()

In [ ]:
print(t_train.shape)
batch_size = 200
total_correct = 0

running_loss = 0.0
accuracy = 0.0
test_size = 1000
test_data_idx = np.random.choice(x_test.shape[0], test_size)
for i in tqdm(test_data_idx):
    x = x_test[i].reshape(1, 1, 28, 28)
    labels = t_test[i]

    outputs = model.forward(x)
    labels = labels.reshape(outputs.shape)
    c = (np.argmax(labels, 1) == np.argmax(outputs, 1)).squeeze()
    total_correct += np.sum(c)

print(f"Accuracy:{total_correct/test_size}")
